In [2]:
import time
import overpy
import concurrent.futures
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def fetch_nearby_addresses(lat, lon, radius=500):
    api = overpy.Overpass()
    query = f"""
    [out:json];
    (
      way["building"](around:{radius},{lat},{lon});
    );
    out center;
    """
    result = api.query(query)
    addresses = []
    for way in result.ways:
        tags = way.tags
        if "addr:street" in tags and "addr:housenumber" in tags:
            address = f'{tags["addr:housenumber"]} {tags["addr:street"]} Melbourne VIC'
            addresses.append(address)
    return addresses


def check_single_address(addr):
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-gpu")

    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 20)

    try:
        driver.get("https://www.telstra.com.au/internet/5g-home-internet")
        wait.until(EC.presence_of_element_located((By.ID, "tcom-sq-main-input")))

        print(f"\n[🔍] Checking: {addr}")
        input_box = driver.find_element(By.ID, "tcom-sq-main-input")
        input_box.clear()
        input_box.send_keys(addr)
        time.sleep(1)
        input_box.send_keys("\b\b")
        time.sleep(1)
        input_box.send_keys(addr[-2:])
        time.sleep(2)

        wait.until(EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, "#adddress-autocomplete-results li.address-option")
        ))
        suggestions = driver.find_elements(By.CSS_SELECTOR, "#adddress-autocomplete-results li.address-option")
        if suggestions:
            driver.execute_script("arguments[0].click();", suggestions[0])
            time.sleep(3)
        else:
            print("[⚠️] No suggestions found")
            return (addr, False)

        result_header = wait.until(EC.presence_of_element_located(
            (By.CSS_SELECTOR, "h3.tcom-sq__result__header__title"))
        )

        is_available = "eligible for 5G Home Internet" in result_header.text
        if is_available:
            print(f"[✅] 5G is AVAILABLE at: {addr}")
        else:
            print(f"[❌] 5G is NOT available at: {addr}")

        # Step 5: Reset form via 'Edit address' or 'Try another address'
        try:
            try:
                reset_button = wait.until(EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, "div[data-sq-result-header] button[data-tcom-sq-reset]")))
            except:
                reset_button = wait.until(EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, "div.tcom-sq__error__button button[data-tcom-sq-reset]")))

            driver.execute_script("arguments[0].scrollIntoView(true);", reset_button)
            time.sleep(0.5)
            driver.execute_script("arguments[0].click();", reset_button)
            wait.until(EC.visibility_of_element_located((By.ID, "tcom-sq-main-input")))
            time.sleep(1)

        except Exception as e:
            print(f"[⚠️] Couldn't reset form: {e}")

        return (addr, is_available)

    except Exception as e:
        print(f"[⚠️] Error checking {addr}: {e}")
        return (addr, False)

    finally:
        driver.quit()


def run_parallel_check(addresses, workers=3):
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as executor:
        futures = [executor.submit(check_single_address, addr) for addr in addresses]
        for future in concurrent.futures.as_completed(futures):
            results.append(future.result())
    return results


if __name__ == "__main__":
    lat, lon = -37.79278, 144.969006   # 570 Lygon Street, Carlton
    addresses = fetch_nearby_addresses(lat, lon)
    addresses.insert(0, "340 Lygon Street Carlton VIC")  # ✅ known 5G address
    addresses_to_check = addresses[:20]

    results = run_parallel_check(addresses_to_check, workers=10)

    print("\n--- ✅ Final 5G Available Addresses ---")
    for addr, is_available in results:
        if is_available:
            print(addr)



[🔍] Checking: 178 Palmerston Street Melbourne VIC
[🔍] Checking: 180 Palmerston Street Melbourne VIC


[🔍] Checking: 887 Swanston Street Melbourne VIC

[🔍] Checking: 414-422 Lygon Street Melbourne VIC

[🔍] Checking: 550 Lygon Street Melbourne VIC
[🔍] Checking: 510 Lygon Street Melbourne VIC


[🔍] Checking: 225 Canning Street Melbourne VIC

[🔍] Checking: 427-429 Cardigan Street Melbourne VIC

[🔍] Checking: 570 Lygon Street Melbourne VIC

[🔍] Checking: 340 Lygon Street Carlton VIC
[❌] 5G is NOT available at: 225 Canning Street Melbourne VIC
[❌] 5G is NOT available at: 887 Swanston Street Melbourne VIC
[❌] 5G is NOT available at: 427-429 Cardigan Street Melbourne VIC
[❌] 5G is NOT available at: 414-422 Lygon Street Melbourne VIC
[❌] 5G is NOT available at: 550 Lygon Street Melbourne VIC
[❌] 5G is NOT available at: 180 Palmerston Street Melbourne VIC[❌] 5G is NOT available at: 510 Lygon Street Melbourne VIC

[❌] 5G is NOT available at: 178 Palmerston Street Melbourne VIC
[❌] 5G is NOT avai